# Fine Tuning of the convolutional network

In [1]:
%pylab
%matplotlib inline

import glob
import os
import mne
CORPORA_PATH = "~/corpora/sets"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))

def normalize_subject(X):
    mean = X.mean(axis=(0, 2)).reshape(-1, 1)
    std = X.std(axis=(0, 2)).reshape(-1, 1)
    return (X - mean) / std

def load_data(filename, normalize=True):
    data_mne = mne.io.read_raw_eeglab(filename, preload=True, event_id={"0": 1, "1": 2})
    data_mne.filter(0, 20)
    events = mne.find_events(data_mne)
    epochs = mne.Epochs(
        data_mne, events,
        baseline=(None, 0), tmin=-0.1, tmax=0.7)

    epochs.load_data()
    
    ch_names = epochs.ch_names
    
    X = epochs.get_data()[:, :-1]
    y = (events[:, 2] == 2).astype('float')

    if len(events) != len(epochs):
        raise ValueError("Epochs events mismatch")
    if normalize: 
        X = normalize_subject(X)
    X = X[..., np.newaxis]
    
    return X, y 


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras.models import load_model
channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'STI 014']

model = load_model("models/model.h5")

Using TensorFlow backend.
/home/jmperez/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
X, y = load_data(files[143])

Reading /home/jmperez/corpora/sets/PruebasMuseo_358001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.1, stratify=y)

In [8]:


from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

def get_metrics(model, X_test, y_test):
    y_pred = model.predict_classes(X_test)
    y_prob = model.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    accuracy = accuracy_score(y_test, y_pred)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall, 
        "roc_auc": auc
    }
    
    
get_metrics(model, X_test, y_test)

{'accuracy': 0.59999999999999998,
 'precision': 0.26136363636363635,
 'recall': 0.76666666666666672,
 'roc_auc': 0.67777777777777781}

In [9]:
model.layers

Let's fix the first two convolutional layers

In [10]:

for i in range(4):
    model.layers[i].trainable = False



model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy
[(l, "Trainable: {}".format(l.trainable)) for l in model.layers]

[(<keras.layers.convolutional.Conv2D at 0x7f726bbd8ba8>, 'Trainable: False'),
 (<keras.layers.convolutional.Conv2D at 0x7f726bbd8e80>, 'Trainable: False'),
 (<keras.layers.core.Flatten at 0x7f726bbd8e10>, 'Trainable: False'),
 (<keras.layers.core.Dropout at 0x7f726bb9a438>, 'Trainable: False'),
 (<keras.layers.core.Dense at 0x7f726bb99d68>, 'Trainable: True'),
 (<keras.layers.core.Dense at 0x7f726bb4afd0>, 'Trainable: True')]

In [11]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='model.h5', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    X_train, y_train, epochs=30, 
    batch_size=64, class_weight={0:1, 1:6}, validation_split=0.01,
    callbacks=[checkpointer, early_stopping]
)

Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 9s 5ms/step - loss: 1.2138 - acc: 0.5739 - val_loss: 2.1109 - val_acc: 0.5882
Epoch 2/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1923 - acc: 0.5727 - val_loss: 2.2624 - val_acc: 0.5294
Epoch 3/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1663 - acc: 0.6344 - val_loss: 2.1916 - val_acc: 0.4706
Epoch 4/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1410 - acc: 0.6045 - val_loss: 2.5851 - val_acc: 0.6471


In [ ]:
print_metrics(model, X_test, y_test)

In [24]:
from os.path import basename
from keras.callbacks import ModelCheckpoint, EarlyStopping

def fix_layers(model, fixed_layers):
    for i in range(fixed_layers):
        model.layers[i].trainable = False
    
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])
    
def fine_tune(fixed_layers):
    model = load_model("models/model.h5")
    
    fix_layers(model, fixed_layers)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

    model.fit(
        X_train, y_train, epochs=30, 
        batch_size=64, class_weight={0:1, 1:6}, validation_split=0.01,
        callbacks=[early_stopping]
    )
    
    return model

def get_analysis(filename, fixed_layers=4):
    model = load_model("models/model.h5")
    X, y = load_data(files[143])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
    
    ret = {"file" : basename(filename)}
    metrics = {"ft_4_{}".format(k):v for k,v in get_metrics(model, X_test, y_test).items()}
    ret.update(metrics)
    
    model = fine_tune(4)
    
    metrics = {"ft_4_{}".format(k):v for k,v in get_metrics(model, X_test, y_test).items()}
    ret.update(metrics)
    
    model = fine_tune(5)
    
    metrics = {"ft_5_{}".format(k):v for k,v in get_metrics(model, X_test, y_test).items()}
    ret.update(metrics)
    
    return ret

get_analysis(files[100], 4)

Reading /home/jmperez/corpora/sets/PruebasMuseo_358001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped
Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 16s 10ms/step - loss: 1.2140 - acc: 0.5733 - val_loss: 2.0426 - val_acc: 0.5882
Epoch 2/30
1603/1603 [==============================] - 10s 6ms/step - loss: 1.1893 - acc: 0.5964 - val_loss: 2.0736 - val_acc: 0.5882
Epoch 3/30
1603/1603 [==============================] - 10s 7ms/step - loss: 1.1730 - acc: 0.5989 - val_loss: 2.2286 - val_acc: 0.5882
Epoch 4/30
1603/1603 [==============================] - 9s 6ms/step - loss: 1.1407 - acc: 0.6319 - val_loss: 2.0906 - val_acc: 0.5294
T

{'file': 'PruebasMuseo_2089001.set',
 'ft_4_accuracy': 0.58333333333333337,
 'ft_4_precision': 0.23529411764705882,
 'ft_4_recall': 0.66666666666666663,
 'ft_4_roc_auc': 0.68488888888888888,
 'ft_5_accuracy': 0.5444444444444444,
 'ft_5_precision': 0.21111111111111111,
 'ft_5_recall': 0.6333333333333333,
 'ft_5_roc_auc': 0.58000000000000007}

In [25]:
df = [get_analysis(file) for file in files[-15:]]
    

Reading /home/jmperez/corpora/sets/PruebasMuseo_358001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events found
0 projection items activated
Loading data for 1800 events and 104 original time points ...
0 bad epochs dropped
Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 12s 8ms/step - loss: 1.2176 - acc: 0.5933 - val_loss: 2.2124 - val_acc: 0.5294
Epoch 2/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1969 - acc: 0.6357 - val_loss: 2.1834 - val_acc: 0.5294
Epoch 3/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1643 - acc: 0.6245 - val_loss: 2.1758 - val_acc: 0.5294
Epoch 4/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1413 - acc: 0.6332 - val_loss: 2.1897 - val_acc: 0.4118
Epoc

1603/1603 [==============================] - 8s 5ms/step - loss: 1.1189 - acc: 0.6482 - val_loss: 2.4792 - val_acc: 0.5882
Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 4s 3ms/step - loss: 1.2178 - acc: 0.5396 - val_loss: 2.0423 - val_acc: 0.5294
Epoch 2/30
1603/1603 [==============================] - 2s 2ms/step - loss: 1.2128 - acc: 0.5552 - val_loss: 2.0660 - val_acc: 0.5294
Epoch 3/30
1603/1603 [==============================] - 2s 2ms/step - loss: 1.2135 - acc: 0.5415 - val_loss: 2.0774 - val_acc: 0.5294
Epoch 4/30
1603/1603 [==============================] - 2s 2ms/step - loss: 1.2137 - acc: 0.5240 - val_loss: 2.0908 - val_acc: 0.5882
Reading /home/jmperez/corpora/sets/PruebasMuseo_358001.fdt
Reading 0 ... 63231  =      0.000 ...   493.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
1800 events found
Events id: [1 2]
1800 matching events 

/home/jmperez/.pyenv/versions/3.6.0/envs/p300/lib/python3.6/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.377354). Check your callbacks.
  % delta_t_median)


1603/1603 [==============================] - 16s 10ms/step - loss: 1.2171 - acc: 0.5951 - val_loss: 2.0711 - val_acc: 0.6471
Epoch 2/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1949 - acc: 0.6032 - val_loss: 2.0735 - val_acc: 0.6471
Epoch 3/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1665 - acc: 0.6039 - val_loss: 2.2346 - val_acc: 0.5294
Epoch 4/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1407 - acc: 0.6500 - val_loss: 2.0362 - val_acc: 0.6471
Epoch 5/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1148 - acc: 0.6319 - val_loss: 2.1779 - val_acc: 0.4118
Epoch 6/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.0855 - acc: 0.6687 - val_loss: 2.1158 - val_acc: 0.4706
Epoch 7/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.0405 - acc: 0.6812 - val_loss: 2.3102 - val_acc: 0.4118
Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [===

/home/jmperez/.pyenv/versions/3.6.0/envs/p300/lib/python3.6/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.692521). Check your callbacks.
  % delta_t_median)


...

1603/1603 [==============================] - 14s 9ms/step - loss: 1.2269 - acc: 0.5833 - val_loss: 2.0144 - val_acc: 0.6471
Epoch 2/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1882 - acc: 0.5646 - val_loss: 2.1132 - val_acc: 0.5882
Epoch 3/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1644 - acc: 0.6313 - val_loss: 1.9782 - val_acc: 0.7059
Epoch 4/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1327 - acc: 0.5827 - val_loss: 2.4745 - val_acc: 0.5294
Epoch 5/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1126 - acc: 0.6856 - val_loss: 2.0868 - val_acc: 0.5882
Epoch 6/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.0768 - acc: 0.6413 - val_loss: 2.4455 - val_acc: 0.4706
Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 6s 3ms/step - loss: 1.2168 - acc: 0.5546 - val_loss: 2.0693 - val_acc: 0.5294
Epoch 2/30
1603/1603 [====

/home/jmperez/.pyenv/versions/3.6.0/envs/p300/lib/python3.6/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.561301). Check your callbacks.
  % delta_t_median)


1603/1603 [==============================] - 14s 9ms/step - loss: 1.2262 - acc: 0.6064 - val_loss: 2.0875 - val_acc: 0.6471
Epoch 2/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1880 - acc: 0.5745 - val_loss: 2.2027 - val_acc: 0.5294
Epoch 3/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1666 - acc: 0.6114 - val_loss: 2.1175 - val_acc: 0.5294
Epoch 4/30
1603/1603 [==============================] - 9s 6ms/step - loss: 1.1377 - acc: 0.6120 - val_loss: 2.2781 - val_acc: 0.3529
Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 5s 3ms/step - loss: 1.2193 - acc: 0.5639 - val_loss: 2.0644 - val_acc: 0.5294
Epoch 2/30
1603/1603 [==============================] - 2s 2ms/step - loss: 1.2109 - acc: 0.5483 - val_loss: 2.0822 - val_acc: 0.5294
Epoch 3/30
1603/1603 [==============================] - 2s 2ms/step - loss: 1.2041 - acc: 0.5459 - val_loss: 2.0861 - val_acc: 0.5294
Epoch 4/30
1603/1603 [====

/home/jmperez/.pyenv/versions/3.6.0/envs/p300/lib/python3.6/site-packages/keras/callbacks.py:116: UserWarning: Method on_batch_end() is slow compared to the batch update (0.610688). Check your callbacks.
  % delta_t_median)


1603/1603 [==============================] - 21s 13ms/step - loss: 1.2107 - acc: 0.5702 - val_loss: 2.1541 - val_acc: 0.5882
Epoch 2/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1874 - acc: 0.5901 - val_loss: 2.2216 - val_acc: 0.4706
Epoch 3/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1650 - acc: 0.6369 - val_loss: 2.1808 - val_acc: 0.5294
Epoch 4/30
1603/1603 [==============================] - 8s 5ms/step - loss: 1.1337 - acc: 0.6276 - val_loss: 2.2514 - val_acc: 0.4706
Train on 1603 samples, validate on 17 samples
Epoch 1/30
1603/1603 [==============================] - 6s 4ms/step - loss: 1.2205 - acc: 0.5515 - val_loss: 2.1062 - val_acc: 0.5294
Epoch 2/30
1603/1603 [==============================] - 3s 2ms/step - loss: 1.2173 - acc: 0.5558 - val_loss: 2.1123 - val_acc: 0.5294
Epoch 3/30
1603/1603 [==============================] - 3s 2ms/step - loss: 1.2139 - acc: 0.5583 - val_loss: 2.0831 - val_acc: 0.5294
Epoch 4/30
1603/1603 [===